# Import data and libraries

In [1]:
from rfgap import RFGAP
import numpy as np

#Data
from aeon.datasets import load_gunpoint
from aeon.datasets import load_basic_motions

#Adjust the path
import sys
sys.path.insert(0, '/yunity/arusty/PF-GAP')

In [2]:
# Load Data
# XTRAIN, YTRAIN = load_gunpoint(split="TRAIN")
# XTEST, YTEST = load_gunpoint(split="TEST")

XTRAIN, YTRAIN = load_basic_motions(split="TRAIN")
XTEST, YTEST = load_basic_motions(split="TEST")

#Generate random static variables to test algorithm
XSTATIC = np.random.rand(XTRAIN.shape[0], 45)
XSTATIC_TEST = np.random.rand(XTEST.shape[0], 45)


# Basic Model Tests 
To ensure they are loaded correctly

## Testing RF_GAP seperately

In [8]:
prediction_type = 'classification'

rf = RFGAP(prediction_type = prediction_type)
rf.fit(np.random.rand(5, 5), np.random.randint(0, 2, size=(5,)))

print(rf.get_proximities().todense())

[[0.         0.31565657 0.44191919 0.13131313 0.11111111]
 [0.18857143 0.         0.24       0.22142857 0.35      ]
 [0.20222222 0.19777778 0.         0.35277778 0.24722222]
 [0.18903509 0.175      0.26754386 0.         0.36842105]
 [0.2712963  0.15       0.18981481 0.38888889 0.        ]]


## Testing the Rockets
Safety first, right?

In [9]:
from aeon.transformations.collection.convolution_based import MultiRocket, MiniRocket
from aeon.datasets import load_unit_test
X_train, y_train = load_unit_test(split="train")
X_test, y_test = load_unit_test(split="test")
rocket = MultiRocket(n_kernels=512)
rocket.fit(X_train)

X_train = rocket.transform(X_train)
X_test = rocket.transform(X_test)

# RFGAP ROCKET

In [10]:
from RFGAP_Rocket import RFGAP_Rocket

#NOT STATIC 
rf_rocket = RFGAP_Rocket(prediction_type = prediction_type, rocket = "Multi",
                         n_kernels=512) # Rocket Kwargs
rf_rocket.fit(XTRAIN, YTRAIN)
prox = rf_rocket.get_proximities().todense()

print(f"Proximities shape {prox.shape}")
print(f"Data shape {XTRAIN.shape}")
print(f"Proximities: \n{prox}")

Proximities shape (40, 40)
Data shape (40, 6, 100)
Proximities: 
[[0.         0.08099856 0.11292086 ... 0.         0.         0.        ]
 [0.08567909 0.         0.09673773 ... 0.         0.         0.        ]
 [0.09607549 0.07463794 0.         ... 0.         0.         0.        ]
 ...
 [0.0025974  0.00779221 0.00519481 ... 0.         0.08857349 0.09839208]
 [0.         0.         0.         ... 0.06691004 0.         0.06258966]
 [0.         0.         0.         ... 0.14338072 0.1078044  0.        ]]


In [6]:
from RFGAP_Rocket import RFGAP_Rocket

rf_rocket = RFGAP_Rocket(prediction_type = prediction_type, rocket = "Multi",
                         n_kernels=512) # Rocket Kwargs
rf_rocket.fit(XTRAIN, YTRAIN, XSTATIC)
prox_static = rf_rocket.get_proximities().todense()

print(f"Proximities shape {prox.shape}")
print(f"Data shape {XTRAIN.shape}")
print(f"Proximities: \n{prox}")

Proximities shape (40, 40)
Data shape (40, 6, 100)
Proximities: 
[[0.         0.08372075 0.1101256  ... 0.         0.         0.        ]
 [0.10414743 0.         0.07812503 ... 0.0034965  0.00227273 0.00174825]
 [0.12003442 0.08393177 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.11745567 0.09141797]
 [0.         0.         0.         ... 0.08982676 0.         0.13285532]
 [0.         0.         0.         ... 0.09388402 0.13054129 0.        ]]


In [7]:
prox_static == prox

matrix([[ True, False, False, ...,  True,  True,  True],
        [False,  True, False, ..., False, False, False],
        [False, False,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True, False, False],
        [False, False, False, ..., False,  True, False],
        [ True,  True,  True, ..., False, False,  True]])